In [2]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving InRisk_Labs_Assignment.xlsx to InRisk_Labs_Assignment.xlsx


In [35]:
df = pd.read_excel('InRisk_Labs_Assignment.xlsx')

print(df.head())

# Checking for missing values
df.isnull().values.any()

#Checking the data structure of the provided data
df.info()
print(df.dtypes)

# Getting the summary statistics of the numeric variables
df["Rainfall_mm"].describe()

     Region       Date  Rainfall_mm
0  Region_A 2024-01-01        4.965
1  Region_A 2024-01-02       27.729
2  Region_A 2024-01-03       12.881
3  Region_A 2024-01-04       32.818
4  Region_A 2024-01-05       36.127
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Region       455 non-null    object        
 1   Date         455 non-null    datetime64[ns]
 2   Rainfall_mm  455 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 10.8+ KB
Region                 object
Date           datetime64[ns]
Rainfall_mm           float64
dtype: object


,Rainfall_mm
count,4.550000e+02
mean,4.052024e+06
std,8.643157e+07
min,-6.301700e+01
25%,2.289500e+01
50%,4.730800e+01
75%,7.652750e+01
max,1.843649e+09


In [39]:
# As the max and minimum of the data are unreasonable, the conclusion can be drawn the data has extreme point errors.

# Negative rainfall values removal
df = df[df["Rainfall_mm"] >= 0]

# Capping w.r.t to highest recorded rainfall
df = df[df["Rainfall_mm"] <= 11871]

print(df["Rainfall_mm"].describe())

# As the queries are deterministic rather being any trend analysis, the outliers are not dealt with.

#Creating Another instance for code testing
df2=df

count    452.000000
mean      49.468361
std       29.875669
min        0.081000
25%       23.279500
50%       47.337000
75%       76.492250
max       99.502000
Name: Rainfall_mm, dtype: float64


In [40]:
# Claim Calculation Function Slabs
Claim_Condition = [
    (0, 10, 0),       # No claim for days <= 10
    (11, 30, 100),    # ₹100/day for 11–30 days
    (31, 50, 200),    # ₹200/day for 31–50 days
    (51, float('inf'), 300)  # ₹300/day for days >= 51
]

# Function for Calculating Claims
def calculate_claim(days):
    for min_days, max_days, rate in Claim_Condition:
        if min_days <= days <= max_days:
            return days * rate
    return 0

# Flag for excess rainfall days
df['Excess_Rainfall'] = df['Rainfall_mm'] > 60

# Group by region and calculating excess rainfall days
excess_rainfall_count = (
    df.groupby('Region')['Excess_Rainfall']
    .sum()
    .reset_index(name='Excess_Rainfall')
)

# Calculating claims for each region
excess_rainfall_count['Claim_Amount'] = excess_rainfall_count['Excess_Rainfall'].apply(calculate_claim)

# The final results
print(excess_rainfall_count)


     Region  Excess_Rainfall  Claim_Amount
0  Region_A               36          7200
1  Region_B               37          7400
2  Region_C               43          8600
3  Region_D               35          7000
4  Region_E               31          6200


In [41]:
# Number of days with Excess rainfall as rainfall > 60 mm for each region

df2['Excess_Rainfall'] = df2['Rainfall_mm'] > 60


# Group by region and count the number of excess rainfall days

excess_rainfall_count_2 = df2.groupby('Region')['Excess_Rainfall'].sum().reset_index()

excess_rainfall_count_2.rename(columns={'Excess_Rainfall': 'Excess_Rainfall'}, inplace=True)

print(excess_rainfall_count_2)
print("Total Count of excess rainfall days:", excess_rainfall_count_2['Excess_Rainfall'].sum())


     Region  Excess_Rainfall
0  Region_A               36
1  Region_B               37
2  Region_C               43
3  Region_D               35
4  Region_E               31
Total Count of excess rainfall days: 182


In [42]:
# Function to calculate claims based on the given slabs

def Total_claim(days):

    if days <= 10:
        return 0
    elif 11 <= days <= 30:
        return days * 100
    elif 31 <= days <= 50:
        return days * 200
    else:
        return days * 300

# The claim calculation function to the excess rainfall count

excess_rainfall_count_2['Claim_Amount'] = excess_rainfall_count_2['Excess_Rainfall'].apply(Total_claim)

# Displaying the results

print(excess_rainfall_count_2)

     Region  Excess_Rainfall  Claim_Amount
0  Region_A               36          7200
1  Region_B               37          7400
2  Region_C               43          8600
3  Region_D               35          7000
4  Region_E               31          6200
